# RPC Test Workers

Worker functions for RPC cross-process tests. These must be in an importable
module for multiprocessing with spawn context.

In [ ]:
#|default_exp rpc.workers

In [ ]:
#|export
from netrun.rpc.base import ChannelClosed
from netrun.rpc.process import SyncProcessChannel

In [ ]:
#|export
def echo_worker(send_q, recv_q):
    """Echo worker that runs in subprocess."""
    channel = SyncProcessChannel(send_q, recv_q)
    try:
        while True:
            key, data = channel.recv()
            channel.send(f"echo:{key}", data)
    except ChannelClosed:
        pass

In [ ]:
#|export
def compute_worker(send_q, recv_q):
    """Compute worker that runs in subprocess."""
    channel = SyncProcessChannel(send_q, recv_q)
    try:
        while True:
            key, data = channel.recv()
            if key == "square":
                channel.send("result", data * data)
            elif key == "double":
                channel.send("result", data * 2)
    except ChannelClosed:
        pass